## Test

In [ ]:
import os
import sys

sys.path.append("./")

import cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import albumentations as album
import segmentation_models_pytorch as smp

from torch.utils.data import DataLoader
#from dataloaders.datasets import Pathology

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# sdm
from scipy.ndimage import distance_transform_edt as distance
from skimage import segmentation as skimage_seg
from skimage import morphology

# test loader
import itertools

from scipy.ndimage import distance_transform_edt

import math

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image

### import data as training

In [ ]:

import segmentation_models_pytorch as smp
ENCODER = 'timm-resnest50d' #'timm-resnest50d' 'densenet201' 'resnet34'
ENCODER_WEIGHTS = None
CLASSES = class_names
ACTIVATION ='softmax2d'  #'identity' # 'softmax2d'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#if torch.cuda.is_available():
#    torch.cuda.set_device(DEVICE)
print(DEVICE)
#ACTIVATION = 'sigmoid'could be None for logits or 'softmax2d' for multiclass segmentation

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation= ACTIVATION,
)

model = model.to(DEVICE)

In [ ]:
micro_iou_score_list = []
micro_f1_score_list = []
micro_accuracy_list = []
micro_recall_list = []
micro_precision_list = []
micro_sensitivity_list = []
micro_specificity_list = []

best_model=model
n=0
best_model.load_state_dict(torch.load('./model/resnest_sdm.pth', map_location=DEVICE))
with torch.no_grad(): 
    best_model.eval()
    total=len(test_loader)
    for i, data in enumerate(itertools.islice(test_loader, total-1)):

        inputs = data['img']['image']

        labels = data['label'] 

        inputs, labels = inputs.float().to(DEVICE), labels.float().to(device=DEVICE, dtype=torch.int64) 
        preds = best_model(inputs)    
        output = F.softmax(preds, dim=1)
 
        target = labels


        _, output = torch.max(output, 1)

        tp, fp, fn, tn = smp.metrics.get_stats(output, target, mode='multilabel', threshold=0.5)        
        
        micro_iou_score = round(smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item(), 3)
        micro_f1_score = round(smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro").item(), 3)
        micro_accuracy = round(smp.metrics.accuracy(tp, fp, fn, tn, reduction="micro").item(),3)
        micro_recall = round(smp.metrics.recall(tp, fp, fn, tn, reduction="micro").item(), 3)
        micro_precision = round(smp.metrics.precision(tp, fp, fn, tn, reduction="micro").item(), 3)
        micro_sensitivity = round(smp.metrics.sensitivity(tp, fp, fn, tn, reduction="micro").item(), 3)
        micro_specificity = round(smp.metrics.specificity(tp, fp, fn, tn, reduction="micro").item(), 3)

        micro_iou_score_list.append(micro_iou_score)
        micro_f1_score_list.append(micro_f1_score)
        micro_accuracy_list.append(micro_accuracy)
        micro_recall_list.append(micro_recall)
        micro_precision_list.append(micro_precision)
        micro_sensitivity_list.append(micro_sensitivity)
        micro_specificity_list.append(micro_specificity)


In [ ]:
fig = plt.figure(figsize=(5,3))
plt.title('micro_iou_score')
plt.plot(micro_iou_score_list,'r-',label='IoU_score')
plt.grid(color = 'gray', linestyle = ':', linewidth = 0.5)


fig = plt.figure(figsize=(5,3))
plt.title('micro_f1_score')
plt.plot(micro_f1_score_list,'b-',label='f1_score')
plt.grid(color = 'gray', linestyle = ':', linewidth = 0.5)

fig = plt.figure(figsize=(5,3))
plt.title('micro_accuracy')
plt.plot(micro_accuracy_list,'y-',label='accuracy')
plt.grid(color = 'gray', linestyle = ':', linewidth = 0.5)

In [ ]:
def metrics_average(metrics_list):
    metric_avg = sum([x for x in metrics_list if isinstance(x, (int, float)) and not math.isnan(x)])/len(metrics_list)
    return round(metric_avg, 3)

In [ ]:
print("micro_iou_score: ",metrics_average(micro_iou_score_list))
print("micro_f1_score_score: ",metrics_average(micro_f1_score_list))
print("micro_accuracy_score: ",metrics_average(micro_accuracy_list))
print("micro_recall_score: ",metrics_average(micro_recall_list))
print("micro_precision_score: ",metrics_average(micro_precision_list))
print("micro_sensitivity_score: ",metrics_average(micro_sensitivity_list))
print("micro_specificity_score: ",metrics_average(micro_specificity_list))

### visualize at once(input, target, prediction, sdm)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
sem_classes = ['background', 'tumor', 'stroma', 'dcis']
sem_class_to_idx = {cls: idx for (idx, cls) in enumerate(sem_classes)} 
tumor_category = sem_class_to_idx["tumor"]
stroma_category = sem_class_to_idx["stroma"]
dcis_category = sem_class_to_idx["dcis"]

best_model = model
best_model.load_state_dict(torch.load('./[BCSS]segmentation_model/sdm_semi_unet_resnest50.pth', map_location=DEVICE))
def visualize_one_sample(test_x_data, test_y_data, model):

    model.eval() 
    with torch.no_grad():
        
        X = test_x_data.float().to(DEVICE)
        Y = test_y_data
        prediction = model(X)  # 예: output shape=(B, num_classes, H, W)
        #sdm
        prediction = F.softmax(prediction, dim=1)
        
      
        _, pred_mask = torch.max(prediction, 1)  # shape=(B, H, W)
        pred_mask = pred_mask[0].cpu().numpy()
        target_mask = Y[0].cpu().numpy().squeeze()

       
        input_img = X[0].cpu().numpy()
        if input_img.ndim == 3 and input_img.shape[0] in [1, 3]:
            if input_img.shape[0] == 1:
                input_img = input_img.squeeze(0)
                cmap_input = 'viridis' 
            else:
                input_img = np.transpose(input_img, (1, 2, 0))
                cmap_input = None 
        else:
            cmap_input = 'viridis'
        
      
        # binary_mask = ((pred_mask == tumor_category) | (pred_mask == stroma_category)).astype(np.uint8)
        binary_mask = np.where(np.isin(pred_mask, [0, 1, 2]), pred_mask, 0)

        # sdm = compute_sdm(pred_mask).squeeze()
        # sdm = torch.from_numpy(sdm).float()
        
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    
    axs[0].imshow(input_img, cmap=cmap_input)
    axs[0].set_title("Input")
    axs[0].axis("off")
    
    
    axs[1].imshow(target_mask, cmap='viridis') 
    axs[1].set_title("Target")
    axs[1].axis("off")
    

    axs[2].imshow(pred_mask, cmap='viridis')
    axs[2].set_title("Prediction")
    axs[2].axis("off")
    
    plt.tight_layout()
    plt.show()

In [ ]:
import os
import cv2
import torchvision.transforms.functional as TF
from torchvision.io import read_image


test_x_data_path = ''
test_y_data_path = ''

test_x_data = sorted(os.listdir(test_x_data_path))
test_y_data = sorted(os.listdir(test_y_data_path))

for i in range(50, 52):

   xname = test_x_data[i]
   yname = test_y_data[i]

   x_path = os.path.join(test_x_data_path, xname)
   y_path = os.path.join(test_y_data_path, yname)
    
    
   x = cv2.imread(x_path)
   x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)

   X = torch.Tensor(x)
   X = X /255
    

   y = cv2.imread(y_path)    
   y = cv2.cvtColor(y, cv2.COLOR_BGR2RGB)

   Y = torch.Tensor(y)

   print("x file : ",xname)
   print("y file : ",yname)

    
   X = np.transpose(X, (2, 0, 1))
   Y = np.transpose(Y, (2, 0, 1))
    
   X.unsqueeze_(0)
   Y.unsqueeze_(0)

   visualize_one_sample(X, Y, best_model)

### calculate FLOPs

In [ ]:
from ptflops import get_model_complexity_info
macs, params = get_model_complexity_info(model, input_res=(3, 256, 256), as_strings=True)

print(f'FLOPs: {macs}, Params: {params}')